This example uses a python library for working with an IAE instance.

Find the library documentation here: http://ibm-analytics-engine-python.readthedocs.io/en/latest/

In [ ]:
! pip install --quiet --upgrade git+https://github.com/snowch/ibm-analytics-engine-python@master

In [ ]:
from ibm_analytics_engine import CloudFoundryAPI, CloudFoundryAPI
from ibm_analytics_engine import IAE, IAEServicePlanGuid

In [ ]:
# Download your IBM Cloud API Key and save it in a secure location

cf = CloudFoundryAPI(api_key_filename='/Users/snowch/.apiKeys/csnow.json')

In [ ]:
SPACE_GUID = cf.space_guid(org_name='my_org_name', space_name='my_space_name')
print(SPACE_GUID)

In [ ]:
# If you couldn't find your space guid, uncomment and run the following cell to print
# out all your orgs and spaces

# cf.print_orgs_and_spaces()

In [ ]:
iae = IAE(cf_client=cf)

In [ ]:
iae.clusters(space_guid=SPACE_GUID)

In [ ]:
cluster_guid = iae.create_cluster(service_instance_name='MY_HADOOP_CLUSTER', 
                   service_plan_guid=IAEServicePlanGuid.LITE, 
                   cluster_creation_parameters={
                        "hardware_config": "default",
                        "num_compute_nodes": 1,
                        "software_package": "ae-1.0-spark",
                    }, 
                   space_guid=SPACE_GUID)

In [ ]:
import time

while iae.status(cluster_instance_id=cluster_guid) == 'in progress':
    print('status is "in progress"')
    time.sleep(60)

In [ ]:
import json
vcap = iae.get_or_create_credentials(cluster_instance_guid=cluster_guid)

vcap_formatted = json.dumps(vcap_json, indent=4, separators=(',', ': '))
print(vcap_formatted)

with open('./vcap.json', 'w') as vcap_file:
    vcap_file.write(vcap_formatted)

In [ ]:
iae.delete_cluster(cluster_guid, recursive=True)